In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
# import missingno
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score

from matplotlib import font_manager, rc
font_path = "/content/drive/MyDrive/L1/Notebooks/dataset/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 가을14, 15년도 데이터 전체에 고객마다 라벨값을 추가한 데이터 readdataset/full_data_with_label_FINAL.pkl'
data = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/full_data_with_label_FINAL.pkl')

In [ ]:
data.drop('spent_less_x',axis=1,inplace=True)

In [ ]:
data.head()
data.rename({'spent_less_y':'spent_less'},axis=1,inplace=True)

In [ ]:
data_increase = data[data.spent_less ==0] #증가고객추출
data_increase.reset_index(inplace=True)
data = data[data.spent_less == 1]  #감소고객추출
data.reset_index(inplace=True) 

In [ ]:
data_increase = data_increase[['CUSTNO','PURCHDATE','cloth_cat','hobby_cat','AFFIL']]
df1 = data[['CUSTNO','PURCHDATE','cloth_cat','hobby_cat','AFFIL']]

In [ ]:
data_increase

,CUSTNO,PURCHDATE,cloth_cat,hobby_cat,AFFIL
0,00002,2014-10-22,uncategorized,hobby_pet,B
1,00002,2014-10-22,uncategorized,hobby_pet,B
2,00002,2014-10-17,cloth_etc,uncategorized,A
3,00002,2014-09-14,uncategorized,hobby_golf,A
4,00002,2014-11-22,uncategorized,hobby_golf,A
...,...,...,...,...,...
165852,19369,2015-09-06,uncategorized,hobby_outdoor,B
165853,19369,2015-09-06,uncategorized,hobby_outdoor,B
165854,19369,2015-09-12,cloth_casual,uncategorized,B
165855,19375,2014-11-14,cloth_lingerie,uncategorized,B


In [ ]:
# 증가고객 연관분석
data_increase_1 = data_increase
copy = data_increase.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,16):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
data_increase_1 = copy.copy()
#B제휴사
frequent_itemsets = apriori(data_increase_1, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.3)


#총 107844건의 거래(DATE로 groupby)
# #지지도 0.001수준
#  - 취미-outdoor, media를 같이 샀을 때 취미 child를 살 확률 0.3 (둘을 같이 살 확률 400건)
#  - 취미-스포츠를 샀을때 스포츠 의류를 살 확률 1.00 (건수 160건)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.038129,0.562192,0.014261,0.374027,0.665302,-0.007175,0.699405
1,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.001484,0.007455,0.001484,1.000000,134.134328,0.001473,inf
2,"(cloth_cat_cloth_business, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.003616,0.562192,0.001641,0.453846,0.807280,-0.000392,0.801621
3,"(cloth_cat_cloth_lingerie, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.005109,0.562192,0.002142,0.419238,0.745720,-0.000730,0.753851
4,"(cloth_cat_cloth_men, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.004636,0.562192,0.002188,0.472000,0.839571,-0.000418,0.829182
5,"(hobby_cat_hobby_golf, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.005248,0.562192,0.002161,0.411661,0.732243,-0.000790,0.744143
6,"(cloth_cat_cloth_lingerie, hobby_cat_hobby_golf)",(cloth_cat_cloth_casual),0.003245,0.562192,0.001307,0.402857,0.716583,-0.000517,0.733172
7,"(hobby_cat_hobby_golf, cloth_cat_cloth_men)",(cloth_cat_cloth_casual),0.002541,0.562192,0.001150,0.452555,0.804983,-0.000279,0.799730
8,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_media)",(hobby_cat_hobby_child),0.003616,0.051797,0.001122,0.310256,5.989848,0.000935,1.374718


In [ ]:
data_increase_1

,cloth_cat_cloth_business,cloth_cat_cloth_casual,cloth_cat_cloth_etc,cloth_cat_cloth_kids,cloth_cat_cloth_lingerie,cloth_cat_cloth_men,cloth_cat_cloth_sports,cloth_cat_cloth_women,hobby_cat_hobby_child,hobby_cat_hobby_golf,hobby_cat_hobby_health,hobby_cat_hobby_instex,hobby_cat_hobby_media,hobby_cat_hobby_outdoor,hobby_cat_hobby_pet,hobby_cat_hobby_sports
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107839,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
107840,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
107841,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
107842,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# data_increase_1[(data_increase_1.hobby_cat_hobby_outdoor==1)&(data_increase_1.hobby_cat_hobby_media==1)]

In [ ]:
# 감소고객 연관분석
data5 = df1.copy()
copy = data5.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,16):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
data5 = copy.copy()
#B제휴사
frequent_itemsets = apriori(data5, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.039441,0.563396,0.011846,0.300340,0.533088,-0.010375,0.624023
1,(cloth_cat_cloth_etc),(cloth_cat_cloth_casual),0.114714,0.563396,0.032911,0.286894,0.509222,-0.031719,0.612256
2,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.040209,0.563396,0.014606,0.363253,0.644755,-0.008048,0.685678
3,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.062436,0.563396,0.017259,0.276434,0.490657,-0.017917,0.603406
4,(cloth_cat_cloth_sports),(hobby_cat_hobby_sports),0.008567,0.001894,0.001894,0.221064,116.724713,0.001878,1.281370
5,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.001894,0.008567,0.001894,1.000000,116.724713,0.001878,inf
6,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.006843,0.050510,0.002010,0.293734,5.815397,0.001664,1.344380
7,"(cloth_cat_cloth_business, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.003681,0.563396,0.001519,0.412621,0.732382,-0.000555,0.743309
8,"(cloth_cat_cloth_lingerie, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.005440,0.563396,0.002108,0.387521,0.687830,-0.000957,0.712847
9,"(cloth_cat_cloth_men, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.004217,0.563396,0.002180,0.516949,0.917559,-0.000196,0.903847


In [ ]:
# 증가고객중 제휴사 A
data_increase_A = data_increase[data_increase.AFFIL == 'A'] 
copy = data_increase_A.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,7):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
data_increase_A = copy.copy()
#B제휴사
frequent_itemsets = apriori(data_increase_A, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.062351,0.728165,0.018450,0.295899,0.406363,-0.026952,0.386073
1,(cloth_cat_cloth_etc),(cloth_cat_cloth_casual),0.149340,0.728165,0.047751,0.319747,0.439113,-0.060993,0.399608
2,(cloth_cat_cloth_lingerie),(cloth_cat_cloth_casual),0.072070,0.728165,0.023563,0.326950,0.449005,-0.028916,0.403885
3,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.054855,0.728165,0.021312,0.388506,0.533541,-0.018632,0.444542
4,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.094718,0.728165,0.026265,0.277301,0.380821,-0.042705,0.376138
5,"(cloth_cat_cloth_business, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.005506,0.728165,0.002513,0.456464,0.626869,-0.001496,0.500123
6,"(cloth_cat_cloth_business, cloth_cat_cloth_lin...",(cloth_cat_cloth_casual),0.002658,0.728165,0.001075,0.404372,0.555329,-0.000861,0.456383
7,"(cloth_cat_cloth_business, cloth_cat_cloth_men)",(cloth_cat_cloth_casual),0.001830,0.728165,0.001031,0.563492,0.773852,-0.000301,0.622748
8,"(cloth_cat_cloth_business, hobby_cat_hobby_golf)",(cloth_cat_cloth_casual),0.003661,0.728165,0.001554,0.424603,0.583114,-0.001111,0.472430
9,"(cloth_cat_cloth_lingerie, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.006319,0.728165,0.002862,0.452874,0.621938,-0.001740,0.496840


In [ ]:
#감소고객중 A제휴사
df_A = df1[df1.AFFIL == 'A'] 

copy = df_A.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,7):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
df_A = copy.copy()
#A제휴사
frequent_itemsets = apriori(df_A, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.055414,0.726276,0.017413,0.314235,0.432666,-0.022833,0.399151
1,(cloth_cat_cloth_etc),(cloth_cat_cloth_casual),0.151382,0.726276,0.048203,0.318419,0.438427,-0.061742,0.401602
2,(cloth_cat_cloth_lingerie),(cloth_cat_cloth_casual),0.074619,0.726276,0.023382,0.313351,0.431449,-0.030812,0.398637
3,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.056825,0.726276,0.022126,0.389372,0.536121,-0.019145,0.448266
4,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.098621,0.726276,0.027037,0.274145,0.377467,-0.044590,0.377106
5,"(cloth_cat_cloth_business, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.005560,0.726276,0.002300,0.413706,0.569626,-0.001738,0.466871
6,"(cloth_cat_cloth_business, hobby_cat_hobby_golf)",(cloth_cat_cloth_casual),0.002992,0.726276,0.001129,0.377358,0.519580,-0.001044,0.439617
7,"(cloth_cat_cloth_lingerie, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.007055,0.726276,0.003020,0.428000,0.589308,-0.002104,0.478538
8,"(cloth_cat_cloth_men, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.006590,0.726276,0.003387,0.513919,0.707608,-0.001399,0.563124
9,"(hobby_cat_hobby_golf, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.008960,0.726276,0.003471,0.387402,0.533408,-0.003036,0.446825


In [ ]:
#증가고객 제휴사B
data_increase_B = data_increase[data_increase.AFFIL == 'B'] 
copy = data_increase_B.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()
 

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,15):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
data_increase_B = copy.copy()
#B제휴사
frequent_itemsets = apriori(data_increase_B, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

# 여성의류를 구매했을때 Casual을 구매 0.27수준 300건requent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

# 여성의류를 구매했을때 Casual을 구매 0.27수준 300건   --  증가에만있음
# 취미_건강 물품을 구매햇을 때 취미_pet물품을 구매 0.25 745건   
# 취미_outdoor, 취미_media 구매시 취미_child구매 0.3 390건
# 취미_건강 물품을 구매햇을 때 취미_pet물품을 구매 0.25 745건
# 취미_outdoor, 취미_media 구매시 취미_child구매 0.3 390건

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_women),(cloth_cat_cloth_casual),0.007023,0.267113,0.001947,0.277193,1.037735,0.000071,1.013945
1,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.003943,0.019812,0.003943,1.000000,50.475124,0.003865,inf
2,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.018358,0.133655,0.004509,0.245638,1.837844,0.002056,1.148446
3,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_child)",(hobby_cat_hobby_media),0.012198,0.112340,0.002982,0.244444,2.175926,0.001611,1.174844
4,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_media)",(hobby_cat_hobby_child),0.009610,0.137647,0.002982,0.310256,2.253997,0.001659,1.250251


In [ ]:
data_increase_B[(data_increase_1.hobby_cat_hobby_outdoor==1)&(data_increase_1.hobby_cat_hobby_media==1)]
data_increase_B.cloth_cat_cloth_women.value_counts()

0    40297
1      285
Name: cloth_cat_cloth_women, dtype: int64

In [ ]:
 #감소고객 제휴사B
df_B = df1[df1.AFFIL == 'B']
copy = df_B.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,15):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
df_B = copy.copy()
#B제휴사
frequent_itemsets = apriori(df_B, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)


#증가고객 제휴사 B
# 여성의류를 구매했을때 Casual을 구매 0.27수준 300건   --  증가에만있음
# 취미_건강 물품을 구매햇을 때 취미_pet물품을 구매 0.25 745건   
# 취미_outdoor, 취미_media 구매시 취미_child구매 0.3 390건
# 취미_건강 물품을 구매햇을 때 취미_pet물품을 구매 0.25 745건
# 취미_outdoor, 취미_media 구매시 취미_child구매 0.3 390건

# 감소고객 제휴사 B
# - cloth_cat_cloth_sports 구매 시 hobby_cat_hobby_sports를 구매 : 신뢰도 1 향상도 44.51 건수 212
# - hobby_cat_hobby_sports 구매 시 cloth_cat_cloth_sports를 구매 : 신뢰도 0.2 향상도 44.51 건수 212
# - hobby_cat_hobby_health 구매 시 hobby_cat_hobby_pet를 구매 : 신뢰도 0.29 향상도 2.21
# - hobby_cat_hobby_media, hobby_cat_hobby_outdoor 구매 시 hobby_cat_hobby_child를 구매 : 신뢰도 0.26 향상도 2.21
# - hobby_cat_hobby_child, hobby_cat_hobby_outdoor 구매 시 hobby_cat_hobby_media를 구매 : 신뢰도 0.22 향상도 2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_women),(cloth_cat_cloth_casual),0.007308,0.280913,0.001522,0.208333,0.741631,-0.000530,0.908321
1,(cloth_cat_cloth_sports),(hobby_cat_hobby_sports),0.022463,0.004966,0.004966,0.221064,44.518248,0.004854,1.277427
2,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.004966,0.022463,0.004966,1.000000,44.518248,0.004854,inf
3,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.017942,0.132434,0.005270,0.293734,2.217965,0.002894,1.228384
4,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_child)",(hobby_cat_hobby_media),0.009416,0.112665,0.002131,0.226368,2.009217,0.001071,1.146973
5,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_media)",(hobby_cat_hobby_child),0.008058,0.119551,0.002131,0.264535,2.212733,0.001168,1.197132


In [ ]:
df2 = df1.drop('AFFIL',axis=1)
df2.CUSTNO=df2.CUSTNO.astype(int)

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/cluster_label_df.pkl') #cluster

In [ ]:
df

,CUSTNO,AGEGROUP,GENDER,FREQUENCY,RESIDENCE,TIME,PURCHAMOUNT,CHANNEL,USE_COMP,NUMB_MEM,labels
0,1,9,0,2,1,1,2,0,3,0,4
1,4,9,1,1,1,1,4,0,1,0,4
2,8,9,0,1,1,1,5,0,1,0,4
3,9,9,1,4,2,2,1,0,0,0,4
4,13,9,1,1,2,2,5,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...
8813,19361,6,1,1,2,2,5,0,0,1,3
8814,19364,2,1,1,2,1,5,0,0,1,3
8815,19371,7,1,1,1,2,5,0,0,1,3
8816,19372,4,1,2,2,1,2,0,1,0,2


In [ ]:
df = df[['CUSTNO','labels']] 

In [ ]:
df_1 = df[df.labels==0]
df_2 = df[df.labels==1]
df_3 = df[df.labels==2]
df_4 = df[df.labels==3]
df_5 = df[df.labels==4]

In [ ]:
df_1.drop('labels', axis=1,inplace=True)
df_2.drop('labels', axis=1,inplace=True)
df_3.drop('labels', axis=1,inplace=True)
df_4.drop('labels', axis=1,inplace=True)
df_5.drop('labels', axis=1,inplace=True)



In [ ]:
cluster_1 = pd.merge(df2,df_1,how='inner')
copy = cluster_1.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,16):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
cluster_1 = copy.copy()
#1cluster
frequent_itemsets = apriori(cluster_1, min_support=0.0005, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

#cluster 0 고객
# hobby_ 스포츠를 구매했을 때 cloth_sports 를 구매활 확률이 100%
#취미_건강을 구매했을 때 취미_펫을 구매활 확률 0.3 
#(cloth_cat_cloth_men, cloth_cat_cloth_lingerie)-	(cloth_cat_cloth_casual) 0.55
#


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.026946,0.480330,0.006621,0.245714,0.511554,-0.006322,0.688957
1,(cloth_cat_cloth_etc),(cloth_cat_cloth_casual),0.087998,0.480330,0.019555,0.222222,0.462645,-0.022713,0.668148
2,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.035184,0.480330,0.011548,0.328228,0.683338,-0.005351,0.773581
3,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.035184,0.480330,0.008700,0.247265,0.514782,-0.008200,0.690376
4,(cloth_cat_cloth_sports),(hobby_cat_hobby_sports),0.010085,0.002079,0.002079,0.206107,99.152672,0.002058,1.256997
5,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.002079,0.010085,0.002079,1.000000,99.152672,0.002058,inf
6,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.008546,0.071291,0.002618,0.306306,4.296558,0.002008,1.338788
7,"(cloth_cat_cloth_lingerie, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.002695,0.480330,0.000693,0.257143,0.535347,-0.000601,0.699556
8,"(cloth_cat_cloth_men, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.003234,0.480330,0.001232,0.380952,0.793106,-0.000321,0.839468
9,"(hobby_cat_hobby_outdoor, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.002079,0.480330,0.000616,0.296296,0.616860,-0.000383,0.738479


In [ ]:
cluster_2 = pd.merge(df2,df_2,how='inner')  #2번집단 (labels = 1)
copy = cluster_2.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,16):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
cluster_2 = copy.copy()


In [ ]:
#2cluster
frequent_itemsets = apriori(cluster_2, min_support=0.0005, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)
# # hobby_ 스포츠를 구매했을 때 cloth_sports 를 구매활 확률이 100%
# hobby_건강을 구매했을 때 취미_pet 0.274%
#(cloth_cat_cloth_etc, cloth_cat_cloth_men)	(cloth_cat_cloth_casual)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.025800,0.547526,0.005639,0.218563,0.399183,-0.008487,0.579028
1,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.026264,0.547526,0.006643,0.252941,0.461971,-0.007737,0.605674
2,(cloth_cat_cloth_women),(cloth_cat_cloth_casual),0.004249,0.547526,0.001004,0.236364,0.431694,-0.001322,0.592525
3,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.023406,0.547526,0.005291,0.226073,0.412898,-0.007524,0.584646
4,(cloth_cat_cloth_sports),(hobby_cat_hobby_sports),0.013866,0.003708,0.003708,0.267409,72.119777,0.003656,1.359958
5,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.003708,0.013866,0.003708,1.000000,72.119777,0.003656,inf
6,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.009694,0.075856,0.002665,0.274900,3.623954,0.001930,1.274506
7,"(cloth_cat_cloth_lingerie, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.002974,0.547526,0.000618,0.207792,0.379511,-0.001010,0.571155
8,"(cloth_cat_cloth_men, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.001738,0.547526,0.000657,0.377778,0.689972,-0.000295,0.727190
9,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_child)",(hobby_cat_hobby_media),0.002395,0.036847,0.000502,0.209677,5.690522,0.000414,1.218684


In [ ]:
cluster_3 = pd.merge(df2,df_3,how='inner')  #3번집단 (labels = 2)
copy = cluster_3.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,16):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
cluster_3 = copy.copy()
#1cluster
frequent_itemsets = apriori(cluster_3, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

#취미_건강 - 취미_pet 0.37
#취미_아이 - 취미_media -> 0.2
#취미_cat - 취미_child 0.21
#cloth_cat_cloth_men, cloth_cat_cloth_lingerie)	(cloth_cat_cloth_casual)	 - 0.57%

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.057523,0.667081,0.021194,0.368444,0.552323,-0.017179,0.527142
1,(cloth_cat_cloth_etc),(cloth_cat_cloth_casual),0.153708,0.667081,0.058233,0.378855,0.567930,-0.044302,0.535977
2,(cloth_cat_cloth_kids),(cloth_cat_cloth_casual),0.005831,0.667081,0.001166,0.200000,0.299814,-0.002724,0.416149
3,(cloth_cat_cloth_lingerie),(cloth_cat_cloth_casual),0.088376,0.667081,0.030904,0.349684,0.524201,-0.028050,0.511935
4,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.061301,0.667081,0.026391,0.430521,0.645381,-0.014501,0.584603
5,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.117810,0.667081,0.035771,0.303637,0.455172,-0.042817,0.478083
6,(hobby_cat_hobby_child),(hobby_cat_hobby_media),0.020788,0.019825,0.004361,0.209756,10.580344,0.003948,1.240345
7,(hobby_cat_hobby_media),(hobby_cat_hobby_child),0.019825,0.020788,0.004361,0.219949,10.580344,0.003948,1.255317
8,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.003068,0.019926,0.001141,0.371901,18.663649,0.001080,1.560380
9,"(cloth_cat_cloth_business, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.006490,0.667081,0.002992,0.460938,0.690977,-0.001338,0.617589


In [ ]:
df_4

,CUSTNO
682,1401
853,1721
869,1765
898,1822
944,1906
...,...
8812,19357
8813,19361
8814,19364
8815,19371


In [ ]:
cluster_4 = pd.merge(df2,df_4,how='inner')  #4번집단 (labels = 3)
copy = cluster_4.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,16):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
cluster_4 = copy.copy()
#1cluster
frequent_itemsets = apriori(cluster_4, min_support=0.0005, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

# (hobby_cat_hobby_sports) - (cloth_cat_cloth_sports) 1
# (hobby_cat_hobby_health)	(hobby_cat_hobby_pet) - 0.25
# (cloth_cat_cloth_etc, cloth_cat_cloth_men) -	(cloth_cat_cloth_casual) 0.46

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.017536,0.447167,0.003597,0.205128,0.458728,-0.004244,0.695499
1,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.021583,0.447167,0.006160,0.285417,0.638277,-0.003491,0.773643
2,(cloth_cat_cloth_women),(cloth_cat_cloth_casual),0.004946,0.447167,0.001079,0.218182,0.487920,-0.001133,0.707112
3,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.015198,0.447167,0.003327,0.218935,0.489604,-0.003469,0.707793
4,(cloth_cat_cloth_sports),(hobby_cat_hobby_sports),0.012725,0.002563,0.002563,0.201413,78.586572,0.002530,1.249003
5,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.002563,0.012725,0.002563,1.000000,78.586572,0.002530,inf
6,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.010656,0.065737,0.002743,0.257384,3.915335,0.002042,1.258070
7,"(cloth_cat_cloth_men, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.001169,0.447167,0.000540,0.461538,1.032138,0.000017,1.026689
8,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_child)",(hobby_cat_hobby_media),0.006430,0.085252,0.001349,0.209790,2.460830,0.000801,1.157602
9,"(hobby_cat_hobby_outdoor, hobby_cat_hobby_media)",(hobby_cat_hobby_child),0.006430,0.092086,0.001349,0.209790,2.278191,0.000757,1.148953


In [ ]:
cluster_5 = pd.merge(df2,df_5,how='inner')  #5번집단 (labels = 4)
copy = cluster_5.copy()
copy = pd.get_dummies(copy,columns =['cloth_cat','hobby_cat'])
copy.drop(['hobby_cat_uncategorized','cloth_cat_uncategorized'],axis=1,inplace= True)
copy = copy.groupby(['CUSTNO','PURCHDATE']).sum()

def change_num(x): #장바구니 분석을 위해서는 index의 값이 Boolean 형태여야함
    if x != 0:
        return 1
    else:
        return 0
for i in range(0,16):
    copy.iloc[:,i] = copy.iloc[:,i].apply(change_num)
copy.reset_index(inplace=True)
copy = copy.drop(['CUSTNO','PURCHDATE'],axis=1)
cluster_5 = copy.copy()
#1cluster
frequent_itemsets = apriori(cluster_5, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by = 'support',ascending=False)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

#hobby_cat_hobby_sports)	(cloth_cat_cloth_sports)	 - 1.0
#(hobby_cat_hobby_health)	(hobby_cat_hobby_pet)	 - 0.4
#(cloth_cat_cloth_etc, cloth_cat_cloth_men)	(cloth_cat_cloth_casual)	0.005618	0.644396	0.003170	0.564315	

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cloth_cat_cloth_business),(cloth_cat_cloth_casual),0.064885,0.562071,0.015650,0.241192,0.429113,-0.020820,0.577127
1,(cloth_cat_cloth_etc),(cloth_cat_cloth_casual),0.164058,0.562071,0.034465,0.210075,0.373752,-0.057748,0.554393
2,(cloth_cat_cloth_men),(cloth_cat_cloth_casual),0.040971,0.562071,0.011869,0.289700,0.515414,-0.011159,0.616540
3,(cloth_cat_cloth_women),(cloth_cat_cloth_casual),0.003517,0.562071,0.001143,0.325000,0.578218,-0.000834,0.648783
4,(hobby_cat_hobby_golf),(cloth_cat_cloth_casual),0.082733,0.562071,0.017320,0.209352,0.372465,-0.029181,0.553886
5,(cloth_cat_cloth_sports),(hobby_cat_hobby_sports),0.007473,0.002022,0.002022,0.270588,133.811765,0.002007,1.368195
6,(hobby_cat_hobby_sports),(cloth_cat_cloth_sports),0.002022,0.007473,0.002022,1.000000,133.811765,0.002007,inf
7,(hobby_cat_hobby_health),(hobby_cat_hobby_pet),0.004044,0.045367,0.001407,0.347826,7.667004,0.001223,1.463771
8,"(cloth_cat_cloth_business, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.006154,0.562071,0.002550,0.414286,0.737070,-0.000910,0.747683
9,"(cloth_cat_cloth_lingerie, cloth_cat_cloth_etc)",(cloth_cat_cloth_casual),0.008264,0.562071,0.002726,0.329787,0.586735,-0.001920,0.653417
